# EM Field Analytical, 10: steering



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from impact import Impact

from pmd_beamphysics.units import mec2

import numpy as np
import os

import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [3]:
# locate the drift template
ifile = '../templates/emfield_analytic_steer/ImpactT.in'
os.path.exists(ifile)

True

In [4]:
# calculate gamma*beta
Etot = 6e6 #eV
gamma = Etot/mec2
GB = np.sqrt(gamma**2 -1)
GB


np.float64(11.699046356605859)

# Use Impact's built-in Gaussian particle generator

In [5]:
I = Impact(ifile)
I.header['Np'] = 100000
I.header['Nx'] = 32
I.header['Ny'] = 32
I.header['Nz'] = 32
I.header['Dt'] = 10e-12
I.header['Bcurr'] = 0

I.header['zmu2'] = GB

# set normal and skew quads
I.ele['STEER']


{'description': 'name:STEER',
 'original': '0.2 0 0 113 0.1  1e-3 0 0 10 0.0254/!name:STEER',
 'L': 0.2,
 'type': 'emfield_analytical',
 'zedge': 0.1,
 'escale': 0.001,
 'rf_frequency': 0.0,
 'theta0_deg': 0.0,
 'subtype': 'hkicker',
 'hgap': 0.0254,
 's': 0.30000000000000004,
 'name': 'STEER'}

In [8]:
I.write_input()
!cat {I.path}/ImpactT.in

! Impact-T input file
!Npcol Nprow
1 1
!Dt Ntstep Nbunch
1e-11 1000000 1
!Dim Np Flagmap Flagerr Flagdiag Flagimg Zimage
6 100000 1 0 2 0 0.02
!Nx Ny Nz Flagbc Xrad Yrad Perdlen
32 32 32 1 0.015 0.015 45.0
!Flagdist Rstartflg Flagsbstp Nemission Temission
2 0 0 0 0.0
!sigx(m) sigpx muxpx xscale pxscale xmu1(m) xmu2
0.001 0.0 0.0 1.0 1.0 0.0 0.0
!sigy(m) sigpy muxpy yscale pyscale ymu1(m) ymu2
0.001 0.0 0.0 1.0 1.0 0.0 0.0
!sigz(m) sigpz muxpz zscale pzscale zmu1(m) zmu2
0.0001 0.0 0.0 1.0 1.0 0.0 11.699046356605859
!Bcurr Bkenergy Bmass Bcharge Bfreq Tini
0 1.0 510998.95 -1.0 1000000000.0 0.0
!=================== LATTICE ===================
0 0 0 -5 0.0 0.0 -1000.0 /!name:2d_to_3d_spacecharge
!0.2 0 0 1 0.5 2.98 0.108 0.016 0.0 0.0 0.0 0.0 0.0 /!name:QE04

! Corrector quads: skew and regular. 
! 10: Dipole field for steering:

0.2 0 0 113 0.1 0.001 0.0 0.0 10 0.0254 /!name:STEER

0 0 0 -99 1.0 0.0 1.0 /!name:stop_1


In [10]:
I.path

'/var/folders/2f/l5_mybzs30j4qqvyj98w1_nw0000gn/T/tmpickr6w65'

# Single particle tracking

In [ ]:
# Track
I2 = I.copy()
I2.configure()

In [ ]:
ele = I2.ele['CQ01']
ele

In [ ]:
# Estimate for angle change for a 6 MeV/c momentum particle, offset by 1 mm.  
ele['b1_gradient']*ele['L_effective']*299792458 / 6e6 * .001

In [ ]:
P2 = I2.track1(s=2.2, z0 = 0, x0=0.001, pz0=6e6)
P2.xp

In [ ]:
I2.plot('mean_x')

# Track beam

In [9]:
# Regular and Skew quads
I.run()

At line 751 of file /Users/runner/miniforge3/conda-bld/impact-t_1717611950773/work/src/DataStruct/Data.f90 (unit = 14)
Fortran runtime error: Cannot open file 'rfdata10': No such file or directory

Error termination. Backtrace:
#0  0x104d8bcf7
#1  0x104d8cb07
#2  0x104d8d42f
#3  0x10506ab1b
#4  0x10506ad4f
#5  0x100abad53
#6  0x100b5cdc3
#7  0x100b69ceb


CalledProcessError: Command '['/Users/chrisonian/Code/mambaforge/envs/lume-impact-dev/bin/ImpactTexe']' returned non-zero exit status 2.

In [ ]:
I.output['stats'].keys()

In [ ]:
PI = I.particles['initial_particles']
PF = I.particles['final_particles']
PI['sigma_y']

In [ ]:
# Compare these. 
key1 = 'mean_z'
key2 = 'sigma_x'
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1+f' ({units1})')
plt.ylabel(key2+f' ({units2})')
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles], 
    [I.particles[name][key2] for name in I.particles], color='red')


In [ ]:
# Compare these. 
key1 = 'mean_z'
key2 = 'sigma_x'
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1+f' ({units1})')
plt.ylabel(key2+f' ({units2})')
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles], 
    [I.particles[name][key2] for name in I.particles], color='red')
key2 = 'sigma_y'
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles], 
    [I.particles[name][key2] for name in I.particles], color='green')

In [ ]:
PF.plot('x', 'y')
PF.plot('delta_z', 'delta_pz')